## Kamodofication requirements

To Kamodofy models and data representing physical quantities, we need to define a set of functions representing the interpolation of each physical variable having the following properties:

* A function name and arguments that follows kamodo's [Syntax](../Syntax/) conventions 
* Default arrays for input arguments
* A meta attribute containing:
    * 'units' - physical units of the values returned by the function
    * 'citation' - How the model or data source should be cited
    * 'equation' - LaTeX representation of this model/data source (if available)
    * 'hidden_args' - A list of function arguments that should not be rendered
* A data attribute - The array holding the variable (if available)
* Any docstrings that provide further context

## Kamodofying Model Readers

Model Readers load data from disk (or server) and provide methods for interpolation. We require that for each variable of interest, the model reader should provide at least one interpolation method that satisfies all of the above requirements. Each model reader will:

1. Open/close files
2. Manage state variables
3. Initialize interpolators
4. Kamodofy interpolators
5. Register functions

In [68]:
from scipy.interpolate import RegularGridInterpolator
from kamodo.kamodo import Kamodo, kamodofy

class MyModel(Kamodo): # the only change
    def __init__(self, filename, **kwargs):
        # perform any necessary I/O
        print('opening {}'.format(filename))
        self.filename = filename
        self.missing_value = np.NAN
        
        # store any data needed for interpolation
        self.x = np.linspace(1, 4, 11)
        self.y = np.linspace(4, 7, 22)
        self.z = np.linspace(7, 9, 33)        
        xx, yy, zz = np.meshgrid(self.x, self.y, self.z, indexing='ij', sparse=True)
        self.data = 2 * xx**3 + 3 * yy**2 - zz
        
        # set up the interpolator
        self.interpolator = RegularGridInterpolator((self.x, self.y, self.z), self.data, 
                                                    bounds_error = False,
                                                   fill_value = self.missing_value)

        # Prepare model for function registration
        super(MyModel, self).__init__(**kwargs) 
        

        # Kamodofy the interpolating function
        @kamodofy(  units = 'kg/m^3',
                    citation = "Pembroke et al 2019", 
                    hidden_args = ['indexing'],
                    data = self.data)
        def rho(x = self.x, y = self.y, z = self.z):
            """A function that returns density in [kg/m^3]. Positions are assumed to be unitless"""
            xx, yy, zz = np.meshgrid(x, y, z, indexing = 'xy')
            points = np.array(list(zip(xx.ravel(),yy.ravel(),zz.ravel())))
            result = self.interpolator(points).reshape(zz.shape, order = 'A')
            return result
        
        # Register the function
        self['rho'] = rho 

model = MyModel('myfile.dat')
        

opening myfile.dat


Here the `@kamodofy` decorator handles the provisioning of kamodo-specific metadata. For example, the declared function `rho` now has a `meta` attribute:

In [72]:
model.rho.meta

{'units': 'kg/m^3',
 'citation': 'Pembroke et al 2019',
 'equation': None,
 'hidden_args': ['indexing']}

`@kamodofy` also adds the data attribute, by calling the function with its default parameters:

In [39]:
model.rho.data.shape

(11, 22, 33)

We could also register the model's interpolating method as part of some other Kamodo object, such as another kamodofied model reader or data source:

In [42]:
from kamodo.kamodo import Kamodo
kamodo = Kamodo(rho = model.rho)
kamodo

Kamodo([(rho(x, y, z),
         <function __main__.MyModel.__init__.<locals>.rho(x=array([1. , 1.3, 1.6, 1.9, 2.2, 2.5, 2.8, 3.1, 3.4, 3.7, 4. ]), y=array([4.        , 4.14285714, 4.28571429, 4.42857143, 4.57142857,
       4.71428571, 4.85714286, 5.        , 5.14285714, 5.28571429,
       5.42857143, 5.57142857, 5.71428571, 5.85714286, 6.        ,
       6.14285714, 6.28571429, 6.42857143, 6.57142857, 6.71428571,
       6.85714286, 7.        ]), z=array([7.    , 7.0625, 7.125 , 7.1875, 7.25  , 7.3125, 7.375 , 7.4375,
       7.5   , 7.5625, 7.625 , 7.6875, 7.75  , 7.8125, 7.875 , 7.9375,
       8.    , 8.0625, 8.125 , 8.1875, 8.25  , 8.3125, 8.375 , 8.4375,
       8.5   , 8.5625, 8.625 , 8.6875, 8.75  , 8.8125, 8.875 , 8.9375,
       9.    ]))>),
        (rho,
         <function __main__.MyModel.__init__.<locals>.rho(x=array([1. , 1.3, 1.6, 1.9, 2.2, 2.5, 2.8, 3.1, 3.4, 3.7, 4. ]), y=array([4.        , 4.14285714, 4.28571429, 4.42857143, 4.57142857,
       4.71428571, 4.85714286, 5.    

We can now compose the function with other expressions

In [43]:
kamodo['vol [cm^3]'] = '4/3 * pi * (x**2 + y**2)**(1/2)'
kamodo['mass [g]'] = 'rho*vol'
kamodo

Kamodo([(rho(x, y, z),
         <function __main__.MyModel.__init__.<locals>.rho(x=array([1. , 1.3, 1.6, 1.9, 2.2, 2.5, 2.8, 3.1, 3.4, 3.7, 4. ]), y=array([4.        , 4.14285714, 4.28571429, 4.42857143, 4.57142857,
       4.71428571, 4.85714286, 5.        , 5.14285714, 5.28571429,
       5.42857143, 5.57142857, 5.71428571, 5.85714286, 6.        ,
       6.14285714, 6.28571429, 6.42857143, 6.57142857, 6.71428571,
       6.85714286, 7.        ]), z=array([7.    , 7.0625, 7.125 , 7.1875, 7.25  , 7.3125, 7.375 , 7.4375,
       7.5   , 7.5625, 7.625 , 7.6875, 7.75  , 7.8125, 7.875 , 7.9375,
       8.    , 8.0625, 8.125 , 8.1875, 8.25  , 8.3125, 8.375 , 8.4375,
       8.5   , 8.5625, 8.625 , 8.6875, 8.75  , 8.8125, 8.875 , 8.9375,
       9.    ]))>),
        (rho,
         <function __main__.MyModel.__init__.<locals>.rho(x=array([1. , 1.3, 1.6, 1.9, 2.2, 2.5, 2.8, 3.1, 3.4, 3.7, 4. ]), y=array([4.        , 4.14285714, 4.28571429, 4.42857143, 4.57142857,
       4.71428571, 4.85714286, 5.    

In [44]:
kamodo['f'] = 'rho(r-10)'
kamodo['g'] = 'r*f'
kamodo

Kamodo([(rho(x, y, z),
         <function __main__.MyModel.__init__.<locals>.rho(x=array([1. , 1.3, 1.6, 1.9, 2.2, 2.5, 2.8, 3.1, 3.4, 3.7, 4. ]), y=array([4.        , 4.14285714, 4.28571429, 4.42857143, 4.57142857,
       4.71428571, 4.85714286, 5.        , 5.14285714, 5.28571429,
       5.42857143, 5.57142857, 5.71428571, 5.85714286, 6.        ,
       6.14285714, 6.28571429, 6.42857143, 6.57142857, 6.71428571,
       6.85714286, 7.        ]), z=array([7.    , 7.0625, 7.125 , 7.1875, 7.25  , 7.3125, 7.375 , 7.4375,
       7.5   , 7.5625, 7.625 , 7.6875, 7.75  , 7.8125, 7.875 , 7.9375,
       8.    , 8.0625, 8.125 , 8.1875, 8.25  , 8.3125, 8.375 , 8.4375,
       8.5   , 8.5625, 8.625 , 8.6875, 8.75  , 8.8125, 8.875 , 8.9375,
       9.    ]))>),
        (rho,
         <function __main__.MyModel.__init__.<locals>.rho(x=array([1. , 1.3, 1.6, 1.9, 2.2, 2.5, 2.8, 3.1, 3.4, 3.7, 4. ]), y=array([4.        , 4.14285714, 4.28571429, 4.42857143, 4.57142857,
       4.71428571, 4.85714286, 5.    

In [45]:
kamodo.detail()

,lhs,rhs,symbol,units
"rho(x, y, z)",rho,None,"rho(x, y, z)",kg/m^3
"vol(x, y)",vol,4*pi*sqrt(x**2 + y**2)/3,"vol(x, y)",cm^3
"mass(x, y, z)",mass,"rho(x, y, z)*vol(x, y)/1000","mass(x, y, z)",g
f(r),f,rho(r - 10),f(r),
g(r),g,r*f(r),g(r),


The following lines will save the image to your working directory.

!!! note
    Saving images requires `plotly-orca-1.2.1`, available through conda: ```conda install -c plotly plotly-orca```

In [46]:
import plotly.io as pio

pio.write_image(model.plot(rho = dict(z = model.z.mean())), 'kamodofied_model_1.svg', validate = False)

We use markdown to embed the image into the notebook.
![Kamodofied Density](kamodofied_model_1.svg?4)

Alternative ways to graph:

In [47]:
## uncomment to open interactive plot in the notebook
# from plotly.offline import init_notebook_mode, iplot
# init_notebook_mode(connected = True)
# iplot(kamodo.plot(rho = dict(x = model.x.mean()))) 

In [15]:
# # uncomment to open interactive plot in separate tab
# from plotly.offline import plot
# plot(kamodo.plot(rho = dict(z = 8))) 